# Weaviate Query Agent

Welcome to this demo notebook! Here, we'll walk you through a comprehensive example showcasing the **Weaviate Query Agent** functionality.  
The Query Agent is an intelligent layer that sits on top of your Weaviate vector database, using generative AI to optimize natural language queries and automatically determine the best search strategy.

### How the Query Agent Works

The Query Agent intelligently handles complex queries by:
- **Query Optimization**: Uses generative AI to transform natural language into optimized vector database queries
- **Collection Selection**: Automatically determines which collection(s) to search based on your question
- **Smart Filtering**: Decides when filtering is needed to narrow down results
- **Aggregation Logic**: Determines if aggregation operations should be performed (counts, grouping, etc.)
- **Execution**: Runs the optimized queries against your Weaviate instance

### Dataset Overview

This notebook demonstrates the Query Agent using three diverse collections:
- **Books**: 10,000 books with titles, authors, descriptions, and genres
- **Brands**: 104 clothing brands with descriptions, ratings, and company information  
- **Ecommerce**: 448 fashion items with prices, categories, reviews, and brand associations

All datasets come pre-vectorized with **Snowflake Arctic Embed v2.0** embeddings via Weaviate's embedding service.

### Further Reading

- Learn more about Query Agents in the [official documentation](https://docs.weaviate.io/agents/query)
- Explore the technical implementation in our [Query Agent tutorial](https://docs.weaviate.io/agents/query/tutorial-ecommerce)
- Understand vector databases in the [Weaviate developer docs](https://weaviate.io/developers/weaviate)

To ensure smooth execution and prevent potential conflicts with your global Python environment, we recommend running the code in a virtual environment. Later in this notebook, we'll guide you through setting up this environment and installing the necessary dependencies.

## Libraries/packages used 

The following libraries are used in this notebook:

* [<code style="color:blue;">weaviate-client[agents]:</code>](https://weaviate.io/developers/weaviate/client-libraries/python) A powerful vector database with Query Agent functionality for intelligent search
* [<code style="color:blue;">datasets:</code>](https://huggingface.co/docs/datasets/) Hugging Face datasets library for loading pre-vectorized data
* [<code style="color:blue;">os:</code>](https://docs.python.org/3/library/os.html) Used for environment variable management
* [<code style="color:blue;">dotenv:</code>](https://pypi.org/project/python-dotenv/) Loads environment variables from .env files

The packages mentioned above are already installed on the learning portal to run this exercise. If you would like to run this code on your local machine, use the following commands:

* `pip install "weaviate-client[agents]" datasets python-dotenv`

<a id='TOC'></a>  
## Table of contents  

1. <a href="#Dependencies">Dependencies</a><br>
2. <a href="#Connecting">Connecting to your Weaviate cluster</a><br>
3. <a href="#Collections">Setting up collections and loading data</a><br>
   3.1 <a href="#BrandsCollection">Create and populate Brands collection</a><br>
   3.2 <a href="#EcommerceCollection">Create and populate Ecommerce collection</a><br>
   3.3 <a href="#BooksCollection">Create and populate Books collection</a><br>
4. <a href="#QueryAgent">Creating the Query Agent</a><br>
5. <a href="#ExampleQueries">Example Query Agent interactions</a><br>
   5.1 <a href="#ClassicRAG">Classic RAG search</a><br>
   5.2 <a href="#DatabaseFiltering">Generated database filtering</a><br>
   5.3 <a href="#StatisticalQueries">Statistical and aggregation queries</a><br>
   5.4 <a href="#MultipleQueries">Querying multiple databases</a><br>
   5.5 <a href="#MissingInfo">Identifying missing information</a><br>

<a id='Dependencies'></a>  
## 1. Dependencies
[Back to table of contents](#TOC)

This section initializes the necessary dependencies and sets environment variables for connecting to your Weaviate Cloud instance.

In [92]:
import os

# set environment variables  
os.environ['WEAVIATE_URL'] = ''  # weaviate instance url  
os.environ['WEAVIATE_API_KEY'] = ''  # admin api key  

In [93]:
import dotenv

dotenv.load_dotenv(override=True)

True

<a id='Connecting'></a>  
## 2. Connecting to your Weaviate cluster  
[Back to table of contents](#TOC)  

To interact with our Weaviate cluster, we'll initialize a client object and verify the connection is successful. This connection will be used throughout the notebook for all data operations and Query Agent interactions.

In [94]:
import weaviate

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
)

# check if the connection is successful  
client.is_ready()

True

<a id='Collections'></a>  
## 3. Setting up collections and loading data  
[Back to table of contents](#TOC)  

In this section, we'll create three collections (Brands, Ecommerce, and Books) and populate them with pre-vectorized data from the [Weaviate Agents dataset on Hugging Face](https://huggingface.co/datasets/weaviate/agents). Each collection uses the Snowflake Arctic Embed v2.0 model for consistent vector embeddings.

In [95]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists  
if client.collections.exists("Brands"):  
    client.collections.delete("Brands")  

client.collections.create(
    name="Brands", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["name", "description", "child_brands", "parent_brand"],
    ),

)

print("Successfully created collection: Brands with corrected schema.")

Successfully created collection: Brands with corrected schema.


<a id='BrandsCollection'></a>  
### 3.1 Create and populate Brands collection  
[Back to table of contents](#TOC)  

We'll start by creating the Brands collection with properties for clothing brand information, then populate it with 104 brand records from the Weaviate agents dataset.

In [96]:
# Upload brands data using the Weaviate-recommended streaming approach
from datasets import load_dataset

# Load fresh streaming dataset for upload
brands_dataset = load_dataset("weaviate/agents", "query-agent-brands", split="train", streaming=True)

brands_collection = client.collections.get("Brands")
with brands_collection.batch.fixed_size(batch_size=100) as batch:
    for item in brands_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded brands data using Weaviate streaming method!")

Successfully uploaded brands data using Weaviate streaming method!


In [97]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists  
if client.collections.exists("Ecommerce"):  
    client.collections.delete("Ecommerce")  

client.collections.create(
    name="Ecommerce", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["name", "description", "collection", "category", "subcategory", "brand", "colors", "tags", "reviews"],
    ),
)

print("Successfully created collection: Ecommerce with corrected schema.")

Successfully created collection: Ecommerce with corrected schema.


<a id='EcommerceCollection'></a>  
### 3.2 Create and populate Ecommerce collection  
[Back to table of contents](#TOC)  

Next, we'll create the Ecommerce collection for fashion items with properties including prices, categories, reviews, and brand associations, then load 448 product records.

In [98]:
# Load fresh streaming dataset for upload
ecommerce_dataset = load_dataset("weaviate/agents", "query-agent-ecommerce", split="train", streaming=True)

ecommerce_collection = client.collections.get("Ecommerce")
with ecommerce_collection.batch.fixed_size(batch_size=100) as batch:
    for item in ecommerce_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded ecommerce data using Weaviate streaming method!")

Successfully uploaded ecommerce data using Weaviate streaming method!


In [99]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists  
if client.collections.exists("Books"):  
    client.collections.delete("Books")  

client.collections.create(
    name="Books", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["title", "author", "description", "genres"],
    ),
)

print("Successfully created collection: Books with corrected schema.")

Successfully created collection: Books with corrected schema.


<a id='BooksCollection'></a>  
### 3.3 Create and populate Books collection  
[Back to table of contents](#TOC)  

Finally, we'll create the Books collection with properties for titles, authors, descriptions, and genres, then populate it with 10,000 book records from various genres including mystery, fiction, and non-fiction.

In [100]:
# Load fresh streaming dataset for upload
books_dataset = load_dataset("weaviate/agents", "query-agent-books", split="train", streaming=True)

books_collection = client.collections.get("Books")
with books_collection.batch.fixed_size(batch_size=100) as batch:
    for item in books_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded books data using Weaviate streaming method!")

Successfully uploaded books data using Weaviate streaming method!


<a id='QueryAgent'></a>  
## 4. Creating the Query Agent  
[Back to table of contents](#TOC)  

Now we'll create our Query Agent instance, which will intelligently route queries across our three collections (Brands, Ecommerce, Books) and automatically determine the optimal search strategy.

The output from the Query Agent will include: the original query, the generated answer to the query, the searches performed, the collections searched, filters applied, aggregates completed, source objects pulled from the database that comntributed to the generated answer and any mising information if teh generated answer is incomplete.

In [101]:
from weaviate_agents.query import QueryAgent

# Instantiate a new agent object, and specify the collections to query
qa = QueryAgent(
    client=client, 
    collections=["Brands", "Ecommerce", "Books"],  # Changed from collections parameter name
)


<a id='ExampleQueries'></a>  
## 5. Example Query Agent interactions  
[Back to table of contents](#TOC)  

Let's explore the Query Agent's capabilities through various types of queries. Notice how the agent automatically determines which collections to search, applies filters, and performs aggregations based on the natural language input.

<a id='BookQueries'></a>  
### 5.1 Classic RAG search  
[Back to table of contents](#TOC)  

This Query Agent is able to respond as a classic RAG system. The Query Agent will run an initial generative step that will optimize the original query for a vector database.



In [102]:
# Perform a query
response = qa.run(
    "Are there any books about King Arthur or Knights that I should read?"
)

# Print the response 
# The output will include: theoriginal query, the generated answer to the query, the seraches performed,
# the collections searched, filters applied, aggregates completed and source objects ulled from the database.
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Are there any books about King Arthur or Knights that I should read?                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ There are several notable books about King Arthur and knights that you might find worthwhile to read:           │
│                                                                                                                 │
│ 1. "The Winter King" by Bernard Cornwell - This historical fiction and fantasy novel explores the story of      │
│ Arthur as a warlord trying to unite Britain amid chaos and Saxon invasions, intertwined with romance and magic. │
│                                                                                                                 │
│ 2. "The Once and Future King" by T.H. White - A classic and masterful retelling of the Arthurian legend,        │
│ encompassing all volumes in one book. It brings warmth, charm, and grandeur to the tale of King Arthur.         │
│                                                                                                                 │
│ 3. "A Knight of the Seven Kingdoms" by George R.R. Martin - While not about King Arthur specifically, this      │
│ fantasy collection follows the adventures of a hedge knight and his squire in a richly detailed medieval world, │
│ offering themes of chivalry and knightly exploits.                                                              │
│                                                                                                                 │
│ 4. "A Knight in Shining Armor" by Jude Deveraux - This is a romance and time travel novel involving a           │
│ 16th-century knight, blending historical fiction and fantasy elements.                                          │
│                                                                                                                 │
│ If you are specifically interested in King Arthur or knights, starting with Bernard Cornwell's and T.H. White's │
│ Arthurian tales is highly recommended for their depth and quality. For a broader take on knights and chivalry,  │
│ George R.R. Martin's and Jude Deveraux's works offer engaging narratives as well.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(queries=['King Arthur'], filters=[[]], filter_operators='AND', collection='Books')    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['books about Knights'],                                                                            │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='f64b1619-6d44-4d03-b559-a29df756bc53' collection='Books'                                          │
│  - object_id='dc980c72-5ce8-4a05-8547-07aa6897b1b0' collection='Books'                                          │
│  - object_id='2d90d9f9-0ed3-48ba-8c2a-51e8b9f1beec' collection='Books'                                          │
│  - object_id='d6328910-07cb-4b8a-a533-4c81520faaf2' collection='Books'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 8951 │
│ Output Tokens: │ 409  │
│ Total Tokens:  │ 9360 │
└────────────────┴──────┘

Total Time Taken: 8.95s

<a id='BookQueries'></a>  
### 5.2 Generated database filtering   
[Back to table of contents](#TOC)  

This Query Agent is able determine appropriate filters and apply filtering for database objects returned by the vector database query. This allows for dynamic filtering in RAG systems, a step that is normally a manual process.


In [103]:
# Perform a query
response = qa.run(
    "I am in the mood for a good mystery book, what should I read?"
)

# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I am in the mood for a good mystery book, what should I read?                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ If you're in the mood for a good mystery book, I recommend "The Keepsake" by Tess Gerritsen. It's part of the   │
│ Rizzoli & Isles series and is highly praised for its suspenseful and riveting storytelling. The plot revolves   │
│ around the discovery of a perfectly preserved mummy that turns out to be a modern-day murder victim. The story  │
│ combines elements of forensic investigation, ancient death rituals, and a chilling killer who taunts detective  │
│ Jane Rizzoli and medical examiner Maura Isles. This book is darkly enthralling, relentlessly surprising, and    │
│ showcases Gersritsen at her peak storytelling power, making it an excellent choice for mystery enthusiasts.     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['mystery books'],                                                                                  │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             TextArrayPropertyFilter(                                                                            │
│                 property_name='genres',                                                                         │
│                 operator=<ComparisonOperator.CONTAINS_ANY: 'contains_any'>,                                     │
│                 value=['Mystery']                                                                               │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='4f4b2573-e058-42a5-967c-963796a4da6b' collection='Books'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 6146 │
│ Output Tokens: │ 231  │
│ Total Tokens:  │ 6377 │
└────────────────┴──────┘

Total Time Taken: 7.37s

<a id='StatisticalQueries'></a>  
### 5.3 Statistical and aggregation queries  
[Back to table of contents](#TOC)  

The Query Agent automatically recognizes when aggregation is needed and performs complex statistical operations like counting and grouping data.

In [104]:
# Perform a query
response = qa.run(
    "Which author has the most books in my collection?"
)

# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Which author has the most books in my collection?                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ The author with the most books in your collection is Stephen King, with a total of 57 books. The next prolific  │
│ authors in your collection include William Shakespeare with 39 books, Terry Pratchett with 35 books, and Agatha │
│ Christie with 33 books.                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔭 No Searches Run                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query=None,                                                                                          │
│     groupby_property='author',                                                                                  │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='author',                                                                             │
│             metrics=<TextMetrics.COUNT: 'COUNT'>,                                                               │
│             top_occurrences_limit=None                                                                          │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 3     │
│ Input Tokens:  │ 13917 │
│ Output Tokens: │ 159   │
│ Total Tokens:  │ 14076 │
└────────────────┴───────┘

Total Time Taken: 7.10s

<a id='MultipleQueries'></a>  
### 5.4 Querying multiple databases  
[Back to table of contents](#TOC)  

The Query agent looks at collection descriptions and property data to determine which collection or collections to query for proper context for the query response generation.

In [105]:
# Perform a query
response = qa.run(
    """I am looking to buy a cool new jean jacket or light coat at or below $150. 
    I want a trendy but also timeless look but can also hold up and last me a long time. 
    If possible, I want to purchase something from a new company that might get me a 
    sponsorship for my social media account."""
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I am looking to buy a cool new jean jacket or light coat at or below $150.                                      │
│     I want a trendy but also timeless look but can also hold up and last me a long time.                        │
│     If possible, I want to purchase something from a new company that might get me a                            │
│     sponsorship for my social media account.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ You can find several trendy yet timeless jean jackets and light coats priced at or below $150 from interesting  │
│ new or newer brands that could align with your style and social media aspirations:                              │
│                                                                                                                 │
│ 1. Jean Jackets:                                                                                                │
│ - Garden Dusk Linen Jacket by Loom & Aura ($90): A soft linen jacket in a subtle lavender, perfect for a        │
│ charming but durable piece with a cottagecore aesthetic. It is new in the collection and well-reviewed for      │
│ comfort and style.                                                                                              │
│ - Rustic Wanderer Utility Jacket by Canvas & Co. ($120): This rugged, durable jacket is great for outdoor,      │
│ long-lasting wear, featuring a utility style with multiple pockets, ideal for a unique yet functional look.     │
│ - Rustic Patcher Jacket by Canvas & Co. ($120): Offers a durable, patchwork design that balances sturdiness     │
│ with a stylish, imperfect aesthetic.                                                                            │
│                                                                                                                 │
│ 2. Light Coats:                                                                                                 │
│ - Elysian Quill Coat by Loom & Aura ($150): A timeless, elegantly tailored coat with vintage intellectual style │
│ in white or grey. It's highly rated for quality and style.                                                      │
│ - Ivory Parchment Trench by Canvas & Co. ($150): A classical light trench coat inspired by classical art,       │
│ perfect for a refined yet trendy look.                                                                          │
│ - Rustic Wanderer's Coat by Nova Nest ($120): Rugged and durable with patchwork detailing, it combines function │
│ and style for outdoorsy, trend-conscious wear.                                                                  │
│                                                                                                                 │
│ In terms of brands, Loom & Aura and Canvas & Co. appear multiple times offering new and stylish options within  │
│ your budget. While specific sponsorship information is not available, brands like these that emphasize          │
│ distinctive designs and are relatively new or niche may be interested in social media partnerships.             │
│                                                                                                                 │
│ Therefore, for a cool, trendy but timeless and durable jacket or coat under $150, consider the above options    │
│ primarily from Loom & Aura and Canvas & Co., which are new-ish brands with a fresh aesthetic that might align   │
│ well with your social media sponsorship goals.                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/9 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['jean jacket'],                                                                                    │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             IntegerPropertyFilter(                                                                              │
│                 property_name='price',                                                                          │
│                 operator=<ComparisonOperator.LESS_EQUAL: '<='>,                                                 │
│                 value=150.0                                                                                     │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/9 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['light coat'],                                                                                     │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             IntegerPropertyFilter(                                                                              │
│                 property_name='price',                                                                          │
│                 operator=<ComparisonOperator.LESS_EQUAL: '<='>,                                                 │
│                 value=150.0                                                                                     │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 3/9 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['trendy jean jacket'],                                                                             │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             TextPropertyFilter(                                                                                 │
│                 property_name='subcategory',                                                                    │
│                 operator=<ComparisonOperator.LIKE: 'LIKE'>,                                                     │
│                 value='*jacket*'                                                                                │
│             ),                                                                                                  │
│             IntegerPropertyFilter(                                                                              │
│                 property_name='price',                                                                          │
│                 operator=<ComparisonOperator.LESS_EQUAL: '<='>,                                                 │
│                 value=150.0                                                                                     │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query=None,                                                                                          │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='name',                                                                               │
│             metrics=<TextMetrics.COUNT: 'COUNT'>,                                                               │
│             top_occurrences_limit=None                                                                          │
│         ),                                                                                                      │
│         IntegerPropertyAggregation(property_name='foundation_year', metrics=<NumericMetrics.MAX: 'MAXIMUM'>)    │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Brands'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── ⚠️ Answer is Partial - Missing Information: ───────────────────────────────────╮
│                                                                                                                 │
│ - Specific information about brand sponsorship opportunities or direct contacts for social media sponsorships   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='f6a3eb34-25cb-4ba1-b0a5-69385154ffec' collection='Ecommerce'                                      │
│  - object_id='ba8d9118-747e-433d-973f-083b735798c4' collection='Ecommerce'                                      │
│  - object_id='95e5c428-aab6-4683-9180-2b16f636d3f4' collection='Ecommerce'                                      │
│  - object_id='ac4a5d8b-7c38-40f7-962a-a877bde3161a' collection='Ecommerce'                                      │
│  - object_id='218b5157-edfd-417c-b16f-a34ffc030eed' collection='Ecommerce'                                      │
│  - object_id='ea6d1206-3370-4ea1-b495-7ca99270e56c' collection='Ecommerce'                                      │
│  - object_id='a4c9094b-d825-49a8-81e4-116a1a5b85fb' collection='Ecommerce'                                      │
│  - object_id='0e15c4a6-3578-4d65-935d-7e3be834a907' collection='Ecommerce'                                      │
│  - object_id='ed3b52e0-015f-4b43-8e6c-1a04a7bc146f' collection='Ecommerce'                                      │
│  - object_id='270f6170-da40-4914-8f68-d45211b6a405' collection='Ecommerce'                                      │
│  - object_id='687960d7-730f-41ab-9f42-4d5ced2bcb3d' collection='Ecommerce'                                      │
│  - object_id='c482293c-b1ad-4932-9d58-67877c36d7ca' collection='Brands'                                         │
│  - object_id='f4f8d921-21a1-4026-a503-6ffc4f8129b5' collection='Brands'                                         │
│  - object_id='0d7c4c42-d8f0-41a7-ae7a-67ab27e293a3' collection='Brands'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 13    │
│ Input Tokens:  │ 37439 │
│ Output Tokens: │ 1177  │
│ Total Tokens:  │ 38616 │
└────────────────┴───────┘

Total Time Taken: 12.61s

<a id='MissingInfo'></a>  
### 5.5 Identifying missing information  
[Back to table of contents](#TOC)  

If a partial response is generated for a query the Query Agent will report the missing information not present in the database.

In [106]:
# Perform a query
response = qa.run(
    """I want to buy a new leather jacket but I am vegan and only buy from socially responsible brands."""
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I want to buy a new leather jacket but I am vegan and only buy from socially responsible brands.                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ If you want to buy a new leather jacket but are vegan and prefer to buy from socially responsible brands, there │
│ are options available that align with your values. You might consider jackets made from alternative materials   │
│ like linen, velvet, or synthetic fabrics rather than real leather. For example, the "Garden Dusk Linen Jacket"  │
│ by Loom & Aura is a soft linen jacket, vegan-friendly, and inspired by nature's calming grace. Another option   │
│ is the "Shroud Jacket" by Vivid Verse, which uses ultra-resistant fabric and offers a sleek, protective urban   │
│ style with no animal-derived materials. The "Mystic Forest Velvet Cloak" by Loom & Aura is also an option if    │
│ you like luxurious velvet texture, though it is not leather but a velvet alternative.                           │
│                                                                                                                 │
│ Regarding socially responsible brands, several brands mentioned are known for quality craftsmanship and have    │
│ some level of social responsibility reputation, although specific sustainable or ethical practices were not     │
│ detailed explicitly in the results. These brands include Loom & Aura (which is the parent brand of Nova Nest),  │
│ Nova Nest itself, Velvet Vesper, Velvet Verse, Echo Ember, and Eko & Stitch. Loom & Aura and its related brands │
│ seem to have a heritage dating back to the 1970s and offer collections with a reputation for thoughtful design. │
│                                                                                                                 │
│ While the search results provide vegan material jacket options and some socially responsible brands, explicit   │
│ claims specifically connecting these brands to vegan or cruelty-free practices were not indicated. Therefore,   │
│ to fully satisfy your request, it is recommended to verify directly with the brand regarding their vegan        │
│ certifications and social responsibility policies.                                                              │
│                                                                                                                 │
│ In summary:                                                                                                     │
│ - Consider jackets from brands like Loom & Aura or Vivid Verse that offer vegan-material alternatives such as   │
│ linen or synthetic fabrics.                                                                                     │
│ - These brands have established reputations but check individually about their vegan and social responsibility  │
│ certifications.                                                                                                 │
│ - No direct leather vegan alternatives branded exclusively as vegan leather were listed in the results.         │
│                                                                                                                 │
│ If you want more specific vegan leather alternatives or brands with certifications, additional detailed info is │
│ missing from the search results.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['leather alternative jacket vegan friendly', 'synthetic leather jacket vegan material'],           │
│     filters=[[], []],                                                                                           │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=[                                                                                                   │
│         'socially responsible leather jacket vegan',                                                            │
│         'ethical leather jacket vegan',                                                                         │
│         'sustainable leather jacket vegan',                                                                     │
│         'eco-friendly leather jacket vegan',                                                                    │
│         'socially conscious leather jacket vegan',                                                              │
│         'corporate social responsibility leather jacket vegan'                                                  │
│     ],                                                                                                          │
│     filters=[[], [], [], [], [], []],                                                                           │
│     filter_operators='OR',                                                                                      │
│     collection='Brands'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── ⚠️ Answer is Partial - Missing Information: ───────────────────────────────────╮
│                                                                                                                 │
│ - Explicit confirmation of vegan leather or leather alternative materials used in jackets from socially         │
│ responsible brands                                                                                              │
│ - Detailed information on the social responsibility or sustainability certifications of the mentioned brands    │
│ - Direct statements from brands that they are fully vegan and socially responsible                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='218b5157-edfd-417c-b16f-a34ffc030eed' collection='Ecommerce'                                      │
│  - object_id='462aa203-bb3f-4652-bd47-6cab68bede65' collection='Ecommerce'                                      │
│  - object_id='a9d82b44-390c-4eea-91f5-22ff212fcaaa' collection='Ecommerce'                                      │
│  - object_id='7db6d88d-82ed-477b-8857-a37f9dd8187a' collection='Brands'                                         │
│  - object_id='08ef46a0-5fc5-4924-921a-e2f7af1c2aab' collection='Brands'                                         │
│  - object_id='9701610d-6a48-4506-97b0-4a93a2012e9b' collection='Brands'                                         │
│  - object_id='e9f8fd4a-32e5-4d27-8632-6ddea0b696fb' collection='Brands'                                         │
│  - object_id='f7f4ae40-9046-4754-b257-c71eb46f1c49' collection='Brands'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 5     │
│ Input Tokens:  │ 12156 │
│ Output Tokens: │ 809   │
│ Total Tokens:  │ 12965 │
└────────────────┴───────┘

Total Time Taken: 17.90s

In [107]:
# close the Weaviate client to free up resources
client.close()